In [1]:
from datasets import load_dataset
from transformers import AutoModel
import torch


In [2]:


data = load_dataset("cvejoski/mjp", download_mode="force_redownload", trust_remote_code=True)
data.set_format("torch")
fimmjp = AutoModel.from_pretrained("cvejoski/FIMMJP", trust_remote_code=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
fimmjp = fimmjp.to(device)
fimmjp.eval()




mjp.py:   0%|          | 0.00/7.01k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


6_st_DFR_V=0.zip:   0%|          | 0.00/263k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

FIMMJP(
  (pos_encodings): DeltaTimeEncoding()
  (ts_encoder): RNNEncoder(
    (rnn): LSTM(8, 256, batch_first=True, bidirectional=True)
  )
  (path_attention): MultiHeadLearnableQueryAttention(
    (W_k): Linear(in_features=512, out_features=128, bias=False)
    (W_v): Linear(in_features=512, out_features=128, bias=False)
  )
  (intensity_matrix_decoder): MLP(
    (layers): Sequential(
      (linear_0): Linear(in_features=2049, out_features=128, bias=True)
      (activation_0): SELU()
      (linear_1): Linear(in_features=128, out_features=128, bias=True)
      (activation_1): SELU()
      (output): Linear(in_features=128, out_features=60, bias=True)
    )
  )
  (initial_distribution_decoder): MLP(
    (layers): Sequential(
      (linear_0): Linear(in_features=2049, out_features=128, bias=True)
      (activation_0): SELU()
      (linear_1): Linear(in_features=128, out_features=128, bias=True)
      (activation_1): SELU()
      (output): Linear(in_features=128, out_features=6, bias=True

In [3]:
batch = {k:v.to(device) for k, v in data["train"][:1].items()}

In [4]:
with torch.no_grad():
    output = fimmjp(batch, n_states=6)

In [5]:
output["im"]


tensor([[[3.4626e+00, 3.4522e+00, 5.7782e-04, 9.8314e-03, 8.3289e-09,
          3.5371e-07],
         [3.6464e-04, 5.9288e+01, 5.9283e+01, 4.8525e-03, 5.4153e-06,
          2.9001e-05],
         [8.9806e-01, 3.4793e-01, 1.2508e+00, 4.7744e-03, 2.2733e-05,
          2.0603e-06],
         [2.8851e-06, 9.6457e-05, 3.6419e-03, 3.7414e-03, 1.5209e-07,
          4.1721e-11],
         [1.6863e-10, 8.3066e-09, 2.1724e-04, 2.6417e-08, 5.8246e-04,
          3.6519e-04],
         [1.0451e-09, 1.1478e-11, 1.4451e-06, 4.4093e-07, 3.6526e-11,
          1.8871e-06]]], device='cuda:0')